In [1]:
import nidaqmx as ni
import numpy as np
from hardware.nidaq import sample_clock, analog_output_sweeper
from hardware.api import Scanner

In [2]:
stage = Scanner()

(4, 1) (4, 1) (4, 1)
[[0.5]
 [0.5]
 [0.5]
 [0.5]]


In [3]:
stage.scanLine([
    np.linspace(0,50,6),
    np.ones(6)*0,
    np.ones(6)*50.,
    np.ones(6)*0,
],SecondsPerPoint=.01,add_aom=False)

(4, 1) (4, 1) (4, 6)
[[0.5  0.55 0.6  0.65 0.7  0.75]
 [0.5  0.5  0.5  0.5  0.5  0.5 ]
 [0.5  0.5  0.5  0.5  0.5  0.5 ]
 [0.5  0.5  0.5  0.5  0.5  0.5 ]]
(4, 1) (4, 1) (4, 1)
[[0.5]
 [0.5]
 [0.5]
 [0.5]]


array([ 630.,  540.,  630.,  540.,  270., 1080.])

In [ ]:
stage.ao_task.clear_task()
clk.clear_task()

No task has been created.
No task has been created.


In [15]:
a = np.arange(12).reshape((4,3))
mask = [False, False, False, False]
a[mask] = np.ones((4,3))[mask]


In [6]:
stage.ao_task.task.is_task_done()

True

In [5]:
stage.ao_task.task.timing.samp_timing_type #= ni.constants.SampleTimingType.ON_DEMAND
# clk.task.timing.samp_quant_samp_mode

<SampleTimingType.ON_DEMAND: 10390>